# This notebok is to get the nearby facilities, GDP and population for our targeted health facility 

## Import packages and set up paths

In [1]:
# import packages
import os
import time 
import numpy as np
import pandas as pd
import xarray as xr 
import geopandas as gp
from rasterio.enums import Resampling
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# set input/output data directories
data_in_path = "../2 Raw Data"
data_out_path = "../3 Output Data"

# specify raw data paths
pop_path = os.path.join(data_in_path,"Population/Count/gha_ppp_2015_1km_Aggregated.tiff")
facility_csv_path = os.path.join(data_out_path,"Ghana_healthsites_io.csv")
# final_data_path = os.path.join(data_out_path, "Albania_final_data_2015pop_2022_0704.csv")

## Some updated user-defined functions 

In [3]:
def print_summary(data):
    """
    DESCRIPTION: This is a function to print the sumamry of a xarray DataArray or Dataset
    
    INPUT - 
    data: xarray DataArray or Dataset
    
    OUTPUT - 
    summary statistics of data
    """
    print(f"shape: {data.rio.shape}")
    print(f"resolution: {data.rio.resolution()}")
    print(f"coordinates boundary: {data.rio.bounds()}")
    print(f"CRS: {data.rio.crs}")
    
def square_helper(arr, y, x, mode):
    """
    DESCRIPTION: This is a helper funtion to do mean or sum in square for 2D numpy array
    
    INPUT - 
    arr: 2D numpy array to be averaged
    (y,x): output dimension
    mode: sum or mean, NAs are ignored 

    OUTPUT - 
    Averaged 2D array
    """
    yy, xx = arr.shape
    if mode == "sum":
        vals = np.nansum(arr.reshape(y, yy//y, x, xx//x),(1,3))
    elif mode == "mean":
        vals = np.nanmean(arr.reshape(y, yy//y, x, xx//x),(1,3))
    else:
        raise Exception("Mode is not suppported, please input `sum' or `mean'")
    return vals

def xarray_square_aggregate(df_raw,stride=10,var_name="band_data",mode="mean"):
    """
    DESCRITPION: This is a function to do square aggregation (mean or sum) for a
    xarray dataArray, generating a new xarray dataArray
    
    INPUT -
    df_raw: 2D xarray DataArray
    stride: Size of the non-overlap square aggregation
    var_name: new DataArray variable name
    mode: square aggregation function
    
    OUTPU -
    A new 2D xarray DataArray with non-overlap square aggregation
    """
    nrow, ncol = df_raw.values.squeeze().shape
    new_nrow,new_ncol = int(nrow/stride), int(ncol/stride)
    data = df_raw.values.squeeze()[0:(new_nrow*stride),0:(new_ncol*stride)]
    new_data = square_helper(data, new_nrow, new_ncol, mode)
    new_x = np.mean(df_raw.coords['x'].values[0:(new_ncol*stride)].reshape(-1,stride),axis=1)
    new_y = np.mean(df_raw.coords['y'].values[0:(new_nrow*stride)].reshape(-1,stride),axis=1)
    # create new datarray
    df = xr.DataArray(
        data=new_data,
        dims=["y", "x"],
        coords=dict(
            x= new_x,
            y= new_y,
        ),
    )
    df.name = var_name
    if df_raw.rio.crs is not None:
        df = df.rio.write_crs(df_raw.rio.crs)
    return df

def grid_reproject(df,ref_data,resample_method = Resampling.nearest):
    """
    DESCRIPTION: This is a function to reproject a xarray dataArray to the coordinates
    of another refrence xarray dataArray.
    
    INPUT - 
    df: The xarray DataArray needing to be reprojected to the new grids
    ref_data: The reference xarray DataArray with reference coordinates (the new grids)
    resample_method: resample method
    
    OUTPU -
    A new xarray DataArray reprojected to the coordinates of ref_data 
    """
    df = df.rio.reproject_match(ref_data, resampling = Resampling.nearest, nodata= np.nan)
    df = df.assign_coords({
        "x": ref_data.x,
        "y": ref_data.y,
    })
    return df

In [4]:
def find_grid_index(x,y,longs,lats,x_min=None,x_max=None,y_min=None,y_max=None):
    """
    DESCRIPTION: This function helps to find the index of the longitudes and 
    latitudes in the given longitude/latitude lists that is closest to the 
    given point (x,y) 
    
    INPUT -
    x: latitude of the given point
    y: longtidue of the given point
    longs: reference list of longitudes
    lats: reference list of latitudes
    x_min, x_max: min/max boundaries of longitude
    y_min, y_max: min/max boundaries of latitude
    
    OUTPUT -
    Index of the closest longitude/latitude to point (x,y)
    """
    if x_min is None and x_max is None:
        x_l = longs[0]- (longs[1]-longs[0])
        x_r = longs[-1] + (longs[-1]-longs[-2])
        x_min = min(x_l, x_r)
        x_max = max(x_l, x_r)
    if x < x_min or x > x_max:
        return -1,-1
    
    if y_min is None and y_max is None:
        y_l = lats[0]- (lats[1]-lats[0])
        y_r = lats[-1] + (lats[-1]-lats[-2])
        y_min = min(y_l, y_r)
        y_max = max(y_l, y_r)
    if y < y_min or y > y_max:
        return -1,-1
    
    ncol = np.argmin(abs(longs-x)) 
    nrow = np.argmin(abs(lats-y))
    if nrow<0 or nrow>=(len(lats)-1) or ncol<0 or ncol>=(len(longs)-1):
        return -1, -1
    else:
        return nrow,ncol

def points_in_grid_new(points_df,ref_data,value_colname=None):
    """
    DESCRIPTION: This function finds how many points (and how many non nan points and their sum  
    if a value_colname is provided) lying in the grids centered at the coordinates of the reference data
    
    INPUT -
    points_df: pandas dataframe that needs to be count 
    ref_data: xarray DataArray with reference coordinates
    value_colname: column name for the values in the dataframe
    
    OUTPUT -
    count: How many points are in each grid of the reference coordinate
    non_nan_count: How many non nan points are in each grid of the reference coordinate
    value_sum: The sum of the values of the non nan points within each grid of the reference coordinate
    """
    long_list = ref_data.coords['x'].values
    lat_list = ref_data.coords['y'].values
    long_list = np.append(long_list, long_list[-1]+ref_data.rio.resolution()[0])
    lat_list = np.append(lat_list, lat_list[-1]+ref_data.rio.resolution()[1])
    count = np.zeros(ref_data.values.shape)
    non_nan_count = np.zeros(ref_data.values.shape)
    if value_colname is not None:
        value_sum = np.empty(ref_data.values.shape)
        value_sum[:] = np.nan
        values = points_df[value_colname]
    else:
        value_sum = None
    grid_index = points_df.apply(lambda row: find_grid_index(row['Long'],row['Lat'],long_list,lat_list), axis=1)
    for j, x in enumerate(grid_index):
        if x[0]!=-1 and x[1]!=-1:
            count[x[0],x[1]] += 1
            if value_colname is not None and ~np.isnan(values[j]):
                non_nan_count[x[0],x[1]] = non_nan_count[x[0],x[1]]+1 
                value_sum[x[0],x[1]] = np.nansum([value_sum[x[0],x[1]],values[j]])
    return count, non_nan_count, value_sum

def spatial_aggregate(count_in,value_sum_in,block_size,agg_mode='sum', non_nan_count_in=None):
    """
    DESCRIPTION: This function apply square sum aggregation with stride 1 for 2D numpy array
    with the same padding
    
    INPUT -
    count_in: 2D numpy array for the count at each grid
    value_sum_in: 2D numpy array for the sum of values at each grid
    block_size: the size of the square sum aggregation
    agg_mode: aggregation operation mode "sum" or "mean"
    non_nan_count_in: 2D array for the non nan count at each grid, only needed for "mean" mode
    
    
    OUTPUT - 
    a new 2D numpy array with square sum aggreation in the same size
    """
    if block_size%2 != 1:
        raise Exception('block_size must be an odd integer!')
    half = block_size//2

    new_count_in = np.zeros(count_in.shape)
    count_in = np.pad(count_in,half)

    return_value = np.zeros(value_sum_in.shape)
    value_sum_in = np.pad(value_sum_in,half)
    
    if agg_mode == 'mean':
        new_nonnan_count_in = np.zeros(non_nan_count_in.shape)
        non_nan_count_in = np.pad(non_nan_count_in,half)
    elif agg_mode == 'sum':
        new_nonnan_count_in = None
    else:
        raise Exception('Error: The function only supports agg_mode for sum or mean.')
    for nrow in range(half,count_in.shape[0]-half):
        for ncol in range(half,count_in.shape[1]-half):
            new_count_in[nrow-half,ncol-half] = np.sum(count_in[(nrow-half):(nrow+half+1),(ncol-half):(ncol+half+1)])
            if agg_mode == 'mean':
                new_nonnan_count_in[nrow-half,ncol-half] = np.sum(non_nan_count_in[(nrow-half):(nrow+half+1),(ncol-half):(ncol+half+1)])
            return_value[nrow-half,ncol-half] = np.nansum(value_sum_in[(nrow-half):(nrow+half+1),(ncol-half):(ncol+half+1)])
    if agg_mode == 'mean':
        new_nonnan_count_in = np.where(new_nonnan_count_in == 0, np.nan, new_nonnan_count_in)
        return_value = return_value/new_nonnan_count_in
        return new_count_in, return_value, new_nonnan_count_in
    else:
        return new_count_in, return_value, new_nonnan_count_in

def layer_aggregate_helper(count_in,value_sum_in,block_size,non_nan_count_in):
    """
    DESCRIPTION: This function apply square sum aggregation with stride 1 for 2D numpy array
    with the same padding
    
    INPUT -
    count_in: 2D numpy array for the count at each grid
    value_sum_in: 2D numpy array for the sum of values at each grid
    block_size: the size of the square sum aggregation
    non_nan_count_in: 2D array for the non nan count at each grid
    
    
    OUTPUT - 
    a new 2D numpy array with square sum aggreation in the same size
    """
    if block_size%2 != 1:
        raise Exception('block_size must be an odd integer!')
    half = block_size//2

    new_count_in = np.zeros(count_in.shape)
    count_in = np.pad(count_in,half)

    return_value = np.zeros(value_sum_in.shape)
    value_sum_in = np.pad(value_sum_in,half)
    
    new_nonnan_count_in = np.zeros(non_nan_count_in.shape)
    non_nan_count_in = np.pad(non_nan_count_in,half)
  
    for nrow in range(half,count_in.shape[0]-half):
        for ncol in range(half,count_in.shape[1]-half):
            new_count_in[nrow-half,ncol-half] = np.sum(count_in[(nrow-half):(nrow+half+1),(ncol-half):(ncol+half+1)])
            new_nonnan_count_in[nrow-half,ncol-half] = np.sum(non_nan_count_in[(nrow-half):(nrow+half+1),(ncol-half):(ncol+half+1)])
            return_value[nrow-half,ncol-half] = np.nansum(value_sum_in[(nrow-half):(nrow+half+1),(ncol-half):(ncol+half+1)])
    return_value = np.where(new_nonnan_count_in == 0, np.nan, return_value)
    return new_count_in, return_value, new_nonnan_count_in

def layer_aggregate(value1, count1, value2, count2, mode = 'sum'):
    value1 = np.array(value1)
    value2 = np.array(value2)
    count1 = np.array(count1)
    count2 = np.array(count2)
    value2[np.where(count1==count2)]=value1[np.where(count1==count2)]
    value1 = np.where(np.isnan(value1), 0, value1)
    value2 = np.where(np.isnan(value2), 0, value2)
    value_diff = value2-value1
    count_diff = count2-count1
    if mode == 'sum':
        return_value = value_diff
        return_value = np.where(count_diff == 0,np.nan,return_value)
    elif mode == 'mean':
        count_diff = np.where(count_diff == 0, np.nan, count_diff)
        return_value = value_diff/count_diff
    else:
        raise Exception('Only support sum or mean for mode')
    return return_value

def print_summary(data):
    """
    DESCRIPTION: This is a function to print the summary of a xarray DataArray or Dataset
    
    INPUT - 
    data: xarray DataArray or Dataset
    
    OUTPUT - 
    summary statistics of data
    """
    print(f"shape: {data.rio.shape}")
    print(f"resolution: {data.rio.resolution()}")
    print(f"coordinates boundary: {data.rio.bounds()}")
    print(f"CRS: {data.rio.crs}")

## Specify reference coordinates/grids

In [5]:
# read in population 
pop_raw = xr.open_dataarray(pop_path)
# take a look at the data
display(pop_raw)
## Clean variables and dimensions
# select band index at 0 
pop = pop_raw.isel(band=0)
# drop extra coordinates dimensions beyond latitude and longtitude
pop = pop.reset_coords(names=['band'],drop=True)
pop.name = 'population'
print_summary(pop)
# make population DataArray as the reference grids
ref_da = pop.copy()
ref_ds = ref_da.to_dataset()
ref_da

<xarray.DataArray 'band_data' (band: 1, y: 773, x: 533)>
[412009 values with dtype=float32]
Coordinates:
  * band         (band) int64 1
  * x            (x) float64 -3.247 -3.239 -3.23 -3.222 ... 1.17 1.178 1.186
  * y            (y) float64 11.17 11.16 11.15 11.15 ... 4.762 4.754 4.745 4.737
    spatial_ref  int64 ...
Attributes:
    STATISTICS_MAXIMUM:  17492.072265625
    STATISTICS_MEAN:     102.50673673986
    STATISTICS_MINIMUM:  0.1342030018568
    STATISTICS_STDDEV:   507.1615317816

shape: (773, 533)
resolution: (0.0083333333, -0.0083333333)
coordinates boundary: (-3.251249987361092, 4.732916852582884, 1.1904166615389087, 11.174583493482885)
CRS: EPSG:4326


<xarray.DataArray 'population' (y: 773, x: 533)>
[412009 values with dtype=float32]
Coordinates:
  * x            (x) float64 -3.247 -3.239 -3.23 -3.222 ... 1.17 1.178 1.186
  * y            (y) float64 11.17 11.16 11.15 11.15 ... 4.762 4.754 4.745 4.737
    spatial_ref  int64 0
Attributes:
    STATISTICS_MAXIMUM:  17492.072265625
    STATISTICS_MEAN:     102.50673673986
    STATISTICS_MINIMUM:  0.1342030018568
    STATISTICS_STDDEV:   507.1615317816

## Facility Data 

In [6]:
var = 'fac'
# read in the clean version of public facilities information
df = pd.read_csv(facility_csv_path,index_col=0)
# remove the target facility as we are looking at everything else nearby
df = df[df.Amenity != 'hospital']
print(f'Total raw facility counts {len(df)}')
display(df.head())

Total raw facility counts 1120


,Facility name,osm_id,Amenity,Ownership,Long,Lat
0,Pro-Life Pharmacy,1590950161,pharmacy,NaN,-0.119871,5.601035
1,Beaver Clinic,1700719794,dentist,private,-0.178916,5.605701
2,Bethel Dental Clinic,1700719799,dentist,NaN,-0.192542,5.616803
3,Adabraka clinic,1728032224,clinic,private,-0.209600,5.566273
4,Iran Clinic,1728032238,doctors,private,-0.212442,5.568037


In [7]:
%%time
# count facilities in each grid
count,non_nan_count, value_sum = points_in_grid_new(df,ref_da)
# counting facilities in 1×1, 3×3, 5×5, and 11×11 grid
merge_ds = ref_ds.copy()
size_list = [1,3,5,11]
for size in size_list:
    new_count, new_value, _ = spatial_aggregate(count,count,size,'sum')
    merge_ds = merge_ds.assign({f"{var}_{size}_cnt" :(["y","x"],new_value)}) # assign new variables 
    print(f"size {size} x {size} done.")

size 1 x 1 done.
size 3 x 3 done.
size 5 x 5 done.
size 11 x 11 done.
CPU times: user 30.7 s, sys: 519 ms, total: 31.2 s
Wall time: 32.2 s


In [8]:
# covert from geospatial xarray dataset to pandas dataframe
merge_ds = merge_ds.assign({f"{var}_1_cnt" :(["y","x"],count)})
merge_df = merge_ds.to_dataframe().reset_index()
merge_df = merge_df.drop(['spatial_ref','population'],axis=1)
merge_df = merge_df.rename({'x':'Long','y':'Lat'},axis=1)
all_sizes = [1,3,5,11]
merge_df = merge_df[['Long','Lat'] + [f"{var}_{size}_cnt" for size in all_sizes]]
for size in all_sizes:
    merge_df[f"{var}_{size}_flag"] = merge_df[f"{var}_{size}_cnt"] > 0 
for idx in range(len(all_sizes)-1):
    size1 = all_sizes[idx]
    size2 = all_sizes[idx+1]
    merge_df[f'{var}_{size1}-{size2}'] = merge_df[f'{var}_{size2}_cnt'] - merge_df[f'{var}_{size1}_cnt']
merge_df

,Long,Lat,fac_1_cnt,fac_3_cnt,fac_5_cnt,fac_11_cnt,fac_1_flag,fac_3_flag,fac_5_flag,fac_11_flag,fac_1-3,fac_3-5,fac_5-11
0,-3.247083,11.170417,0.0,0.0,0.0,0.0,False,False,False,False,0.0,0.0,0.0
1,-3.247083,11.162083,0.0,0.0,0.0,0.0,False,False,False,False,0.0,0.0,0.0
2,-3.247083,11.153750,0.0,0.0,0.0,0.0,False,False,False,False,0.0,0.0,0.0
3,-3.247083,11.145417,0.0,0.0,0.0,0.0,False,False,False,False,0.0,0.0,0.0
4,-3.247083,11.137083,0.0,0.0,0.0,0.0,False,False,False,False,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
412004,1.186250,4.770417,0.0,0.0,0.0,0.0,False,False,False,False,0.0,0.0,0.0
412005,1.186250,4.762084,0.0,0.0,0.0,0.0,False,False,False,False,0.0,0.0,0.0
412006,1.186250,4.753750,0.0,0.0,0.0,0.0,False,False,False,False,0.0,0.0,0.0
412007,1.186250,4.745417,0.0,0.0,0.0,0.0,False,False,False,False,0.0,0.0,0.0


In [9]:
# assign facility count rank for 1x1 grid
count_values = merge_df['fac_5_cnt']
count_levels = np.array(['nan']*len(count_values))
count_levels[~np.isnan(count_values)] = 'C'
count_levels[count_values >= 1] = 'B'
count_levels[count_values >= 2] = 'A'
merge_df['fcr_5'] = count_levels
merge_df['fcr_5'].value_counts()

C    405331
B      3975
A      2703
Name: fcr_5, dtype: int64

In [10]:
merge_df

,Long,Lat,fac_1_cnt,fac_3_cnt,fac_5_cnt,fac_11_cnt,fac_1_flag,fac_3_flag,fac_5_flag,fac_11_flag,fac_1-3,fac_3-5,fac_5-11,fcr_5
0,-3.247083,11.170417,0.0,0.0,0.0,0.0,False,False,False,False,0.0,0.0,0.0,C
1,-3.247083,11.162083,0.0,0.0,0.0,0.0,False,False,False,False,0.0,0.0,0.0,C
2,-3.247083,11.153750,0.0,0.0,0.0,0.0,False,False,False,False,0.0,0.0,0.0,C
3,-3.247083,11.145417,0.0,0.0,0.0,0.0,False,False,False,False,0.0,0.0,0.0,C
4,-3.247083,11.137083,0.0,0.0,0.0,0.0,False,False,False,False,0.0,0.0,0.0,C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
412004,1.186250,4.770417,0.0,0.0,0.0,0.0,False,False,False,False,0.0,0.0,0.0,C
412005,1.186250,4.762084,0.0,0.0,0.0,0.0,False,False,False,False,0.0,0.0,0.0,C
412006,1.186250,4.753750,0.0,0.0,0.0,0.0,False,False,False,False,0.0,0.0,0.0,C
412007,1.186250,4.745417,0.0,0.0,0.0,0.0,False,False,False,False,0.0,0.0,0.0,C


In [11]:
# %%time 
# # save the data to shapefile
# gdf = gp.GeoDataFrame(
#      merge_df, geometry=gp.points_from_xy(merge_df.Long, merge_df.Lat))
# gdf.to_file(os.path.join(data_out_path,f'{var}_2022_06_07.shp') )

In [12]:
%%time 
# save the data to csv
merge_df.to_csv(os.path.join(data_out_path,f'{var}_2022_0824.csv') )

CPU times: user 2.58 s, sys: 103 ms, total: 2.68 s
Wall time: 3 s


## Read in preprocessed data for other variables (GDP, population, nightlight, death)

In [13]:
df = pd.read_csv(os.path.join(data_out_path, "gha_final_data_v0.csv"),index_col=0)
print(df.shape)
df = df.rename({"GDP_PPP":"GDP", "population":"pop","nightlight":"nl"},axis=1)
df

(54018, 6)


,Long,Lat,GDP,pop,nl,pc
0,-3.238750,6.653750,173934.44,189.508070,inf,917.82074
1,-3.230417,6.803750,802710.80,71.738560,inf,11189.39100
2,-3.230417,6.795417,203373.44,64.722330,inf,3142.24540
3,-3.230417,6.787084,158816.89,63.303300,inf,2508.82500
4,-3.230417,6.778750,350244.28,59.484226,inf,5888.01950
...,...,...,...,...,...,...
54013,1.186250,6.145417,623688.06,1289.173200,2.820924,483.78918
54014,1.186250,6.137084,304008.72,1359.127700,2.677600,223.67929
54015,1.186250,6.128750,725776.40,1734.647100,2.861170,418.40002
54016,1.186250,6.120417,7984543.00,2745.931000,6.389873,2907.77270


## Average for GDP

In [14]:
%%time
for var in ["GDP"]:
    print(f"Processing for {var}...")
    # count nan valus
    print(f"{sum(np.isnan(df[var]))} missing values out of total {len(df[var])} values")
    display(df[var].describe())
    time1 = time.time()
    count,non_nan_count, value_sum = points_in_grid_new(df,ref_da,var)
    time2 = time.time()
    print(f"{time2-time1}s for count")
    merge_ds = ref_ds.copy()
    size = 1
    merge_ds = merge_ds.assign({f"{var}_{size}" :(["y","x"],value_sum)})
    merge_ds = merge_ds.assign({f"{var}_{size}_cnt" :(["y","x"],non_nan_count)})
    print(f"{var} size {size} x {size} done.")
    size_list = [3, 5, 11] # 1km, 3km, 5km and 11km
    for size in size_list:
        new_count, new_value, new_nonnan_count = layer_aggregate_helper(count,value_sum,size, non_nan_count)
        merge_ds = merge_ds.assign({f"{var}_{size}" :(["y","x"],new_value)})
        merge_ds = merge_ds.assign({f"{var}_{size}_cnt" :(["y","x"],new_nonnan_count)})
        print(f"{var} size {size} x {size} done.")
    # covert from geospatial xarray dataset to pandas dataframe
    merge_df = merge_ds.to_dataframe().reset_index()
    merge_df = merge_df.drop(['spatial_ref','population'],axis=1)
    merge_df = merge_df.rename({'x':'Long','y':'Lat'},axis=1)
    all_sizes = [1, 3, 5, 11]
    for idx in range(len(all_sizes)-1):
        size1 = all_sizes[idx]
        size2 = all_sizes[idx+1]
        merge_df[f'{var}_{size1}-{size2}'] = layer_aggregate(merge_df[f'{var}_{size1}'],
                                                     merge_df[f'{var}_{size1}_cnt'],
                                                     merge_df[f'{var}_{size2}'], 
                                                     merge_df[f'{var}_{size2}_cnt'],
                                                     'mean')
    for size in all_sizes:
        count  = np.where(merge_df[f'{var}_{size}_cnt']==0, np.nan, merge_df[f'{var}_{size}_cnt'])
        # merge_df[f'{var}_{size}'] = merge_df[f'{var}_{size}']/count
    merge_df = merge_df[['Long','Lat'] +\
                        [f"{var}_{size}" for size in all_sizes] +\
                        [f"{var}_{size}_cnt" for size in all_sizes] +\
                        [f"{var}_{all_sizes[i]}-{all_sizes[i+1]}" for i in range(len(all_sizes)-1)]]
    if var == "GDP":
            # assign gdp rating 
            gdp_values = merge_df[f'{var}_1']
            gdp_cutoff = np.nanquantile(gdp_values,[0.7,0.95])
            gdp_levels = np.array(['nan']*len(gdp_values))
            gdp_levels[~np.isnan(gdp_values)] = 'C'
            gdp_levels[gdp_values > gdp_cutoff[0]] = 'B'
            gdp_levels[gdp_values > gdp_cutoff[1]] = 'A'
            merge_df['gdp_rating'] = gdp_levels
            display(merge_df['gdp_rating'].value_counts())
    display(merge_df.head(15))
    time3 = time.time()
    print(f"{time3-time2}s for aggregation")
#     # save the data to shapefile, which will be large, so saving and reading will be slow
#     gdf = gp.GeoDataFrame(
#          merge_df, geometry=gp.points_from_xy(merge_df.Long, merge_df.Lat))
#     gdf.to_file(os.path.join(data_out_path,f'{var}_2022_06_07.shp') )
    merge_df.to_csv(os.path.join(data_out_path,f'{var}_2022_0824.csv') )
    print(f"{var} data saved")
    time4 = time.time()
    print(f"{time4-time3}s for saving")

Processing for GDP...
0 missing values out of total 54018 values


count    5.401800e+04
mean     1.969608e+06
std      5.959123e+06
min      1.456718e-01
25%      7.207589e+04
50%      3.056103e+05
75%      1.335608e+06
max      2.366937e+08
Name: GDP, dtype: float64

3.082427740097046s for count
GDP size 1 x 1 done.
GDP size 3 x 3 done.
GDP size 5 x 5 done.
GDP size 11 x 11 done.


nan    357991
C       37812
B       13505
A        2701
Name: gdp_rating, dtype: int64

,Long,Lat,GDP_1,GDP_3,GDP_5,GDP_11,GDP_1_cnt,GDP_3_cnt,GDP_5_cnt,GDP_11_cnt,GDP_1-3,GDP_3-5,GDP_5-11,gdp_rating
0,-3.247083,11.170417,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,nan
1,-3.247083,11.162083,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,nan
2,-3.247083,11.153750,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,nan
3,-3.247083,11.145417,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,nan
4,-3.247083,11.137083,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,nan
5,-3.247083,11.128750,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,nan
6,-3.247083,11.120417,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,nan
7,-3.247083,11.112083,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,nan
8,-3.247083,11.103750,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,nan
9,-3.247083,11.095417,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,nan


32.70374393463135s for aggregation
GDP data saved
3.8930323123931885s for saving
CPU times: user 37.3 s, sys: 913 ms, total: 38.2 s
Wall time: 39.7 s


In [15]:
merge_df[merge_df["gdp_rating"]!="nan"]

,Long,Lat,GDP_1,GDP_3,GDP_5,GDP_11,GDP_1_cnt,GDP_3_cnt,GDP_5_cnt,GDP_11_cnt,GDP_1-3,GDP_3-5,GDP_5-11,gdp_rating
1315,-3.238750,6.653750,173934.44,1.207914e+06,1.375855e+06,3.006839e+06,1.0,2.0,3.0,20.0,1.033980e+06,1.679407e+05,9.594028e+04,C
2070,-3.230417,6.803750,802710.80,2.220125e+06,4.145302e+06,3.540515e+07,1.0,5.0,11.0,42.0,3.543536e+05,3.208627e+05,1.008382e+06,C
2071,-3.230417,6.795417,203373.44,2.222219e+06,4.568716e+06,3.729958e+07,1.0,6.0,13.0,44.0,4.037692e+05,3.352138e+05,1.055834e+06,C
2072,-3.230417,6.787084,158816.89,1.749767e+06,4.063282e+06,3.767204e+07,1.0,5.0,12.0,45.0,3.977375e+05,3.305022e+05,1.018447e+06,C
2073,-3.230417,6.778750,350244.28,1.009246e+06,3.280498e+06,3.737013e+07,1.0,4.0,10.0,43.0,2.196672e+05,3.785420e+05,1.033019e+06,C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
411839,1.186250,6.145417,623688.06,7.203467e+06,1.066206e+07,9.106583e+07,1.0,6.0,12.0,40.0,1.315956e+06,5.764314e+05,2.871563e+06,C
411840,1.186250,6.137084,304008.72,5.055792e+06,2.362312e+07,9.670761e+07,1.0,6.0,15.0,42.0,9.503566e+05,2.063037e+06,2.706833e+06,C
411841,1.186250,6.128750,725776.40,1.442248e+07,2.460980e+07,9.670761e+07,1.0,6.0,15.0,42.0,2.739341e+06,1.131924e+06,2.670289e+06,C
411842,1.186250,6.120417,7984543.00,1.604402e+07,2.384695e+07,9.670761e+07,1.0,6.0,13.0,42.0,1.611895e+06,1.114704e+06,2.512437e+06,B


In [16]:
merge_df.isnull().sum()

Long               0
Lat                0
GDP_1         357991
GDP_3         288319
GDP_5         246309
GDP_11        177224
GDP_1_cnt          0
GDP_3_cnt          0
GDP_5_cnt          0
GDP_11_cnt         0
GDP_1-3       289729
GDP_3-5       255415
GDP_5-11      182491
gdp_rating         0
dtype: int64

## Sum for population

In [17]:
%%time
for var in ["pop"]:
    print(f"Processing for {var}...")
    # count nan valus
    print(f"{sum(np.isnan(df[var]))} missing values out of total {len(df[var])} values")
    display(df[var].describe())
    time1 = time.time()
    count,non_nan_count, value_sum = points_in_grid_new(df,ref_da,var)
    time2 = time.time()
    print(f"{time2-time1}s for count")
    merge_ds = ref_ds.copy()
    size = 1
    merge_ds = merge_ds.assign({f"{var}_{size}" :(["y","x"],value_sum)})
    merge_ds = merge_ds.assign({f"{var}_{size}_cnt" :(["y","x"],non_nan_count)})
    print(f"{var} size {size} x {size} done.")
    size_list = [3, 5, 11]
    for size in size_list:
        new_count, new_value, new_nonnan_count = layer_aggregate_helper(count,value_sum,size, non_nan_count)
        merge_ds = merge_ds.assign({f"{var}_{size}" :(["y","x"],new_value)})
        merge_ds = merge_ds.assign({f"{var}_{size}_cnt" :(["y","x"],new_nonnan_count)})
        print(f"{var} size {size} x {size} done.")
    # covert from geospatial xarray dataset to pandas dataframe
    merge_df = merge_ds.to_dataframe().reset_index()
    merge_df = merge_df.drop(['spatial_ref','population'],axis=1)
    merge_df = merge_df.rename({'x':'Long','y':'Lat'},axis=1)
    all_sizes = [1,3,5,11]
    for idx in range(len(all_sizes)-1):
        size1 = all_sizes[idx]
        size2 = all_sizes[idx+1]
        merge_df[f'{var}_{size1}-{size2}'] = layer_aggregate(merge_df[f'{var}_{size1}'],
                                                     merge_df[f'{var}_{size1}_cnt'],
                                                     merge_df[f'{var}_{size2}'], 
                                                     merge_df[f'{var}_{size2}_cnt'],
                                                     'sum')
    merge_df = merge_df[['Long','Lat'] +\
                        [f"{var}_{size}" for size in all_sizes] +\
                        [f"{var}_{size}_cnt" for size in all_sizes] +\
                        [f"{var}_{all_sizes[i]}-{all_sizes[i+1]}" for i in range(len(all_sizes)-1)]]
    display(merge_df.head())
    time3 = time.time()
    print(f"{time3-time2}s for aggregation")
#     # save the data to shapefile, which will be large, so saving and reading will be slow
#     gdf = gp.GeoDataFrame(
#          merge_df, geometry=gp.points_from_xy(merge_df.Long, merge_df.Lat))
#     gdf.to_file(os.path.join(data_out_path,f'{var}_2022_06_07.shp') )
    merge_df.to_csv(os.path.join(data_out_path,f'{var}_2022_0824.csv') )
    print(f"{var} data saved")
    time4 = time.time()
    print(f"{time4-time3}s for saving")

Processing for pop...
0 missing values out of total 54018 values


count    54018.000000
mean       331.774286
std       1115.389666
min          0.284469
25%         62.016001
50%        116.546530
75%        247.480120
max      17492.072000
Name: pop, dtype: float64

3.0778310298919678s for count
pop size 1 x 1 done.
pop size 3 x 3 done.
pop size 5 x 5 done.
pop size 11 x 11 done.


,Long,Lat,pop_1,pop_3,pop_5,pop_11,pop_1_cnt,pop_3_cnt,pop_5_cnt,pop_11_cnt,pop_1-3,pop_3-5,pop_5-11
0,-3.247083,11.170417,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN
1,-3.247083,11.162083,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN
2,-3.247083,11.153750,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN
3,-3.247083,11.145417,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN
4,-3.247083,11.137083,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN


32.17578101158142s for aggregation
pop data saved
3.8012421131134033s for saving
CPU times: user 37 s, sys: 833 ms, total: 37.8 s
Wall time: 39.1 s


In [18]:
merge_df[merge_df["pop_1_cnt"]!=0]

,Long,Lat,pop_1,pop_3,pop_5,pop_11,pop_1_cnt,pop_3_cnt,pop_5_cnt,pop_11_cnt,pop_1-3,pop_3-5,pop_5-11
1315,-3.238750,6.653750,189.508070,525.692120,643.123800,1780.557019,1.0,2.0,3.0,20.0,336.184050,117.431680,1137.433219
2070,-3.230417,6.803750,71.738560,299.271094,674.243431,2915.332408,1.0,5.0,11.0,42.0,227.532534,374.972337,2241.088977
2071,-3.230417,6.795417,64.722330,353.931814,779.988827,3102.263442,1.0,6.0,13.0,44.0,289.209484,426.057013,2322.274615
2072,-3.230417,6.787084,63.303300,293.126470,708.279937,3166.634283,1.0,5.0,12.0,45.0,229.823170,415.153467,2458.354346
2073,-3.230417,6.778750,59.484226,230.593341,585.644452,3015.997644,1.0,4.0,10.0,43.0,171.109115,355.051111,2430.353192
...,...,...,...,...,...,...,...,...,...,...,...,...,...
411839,1.186250,6.145417,1289.173200,9289.100200,15500.678050,52159.083790,1.0,6.0,12.0,40.0,7999.927000,6211.577850,36658.405740
411840,1.186250,6.137084,1359.127700,7623.677800,21511.983250,56415.679690,1.0,6.0,15.0,42.0,6264.550100,13888.305450,34903.696440
411841,1.186250,6.128750,1734.647100,10080.279100,23872.927550,56415.679690,1.0,6.0,15.0,42.0,8345.632000,13792.648450,32542.752140
411842,1.186250,6.120417,2745.931000,13682.134900,22546.197100,56415.679690,1.0,6.0,13.0,42.0,10936.203900,8864.062200,33869.482590


## Merging facility counts, GDP and population onto our target facility file

### read in facility df -- use hospital as an example

In [19]:
fac_df = pd.read_csv(os.path.join(data_out_path, "Ghana_healthsites_io.csv"))
fac_df = fac_df[fac_df.Amenity == 'hospital'].drop("Unnamed: 0", axis=1).reset_index(drop=True)
fac_df.head(5)

,Facility name,osm_id,Amenity,Ownership,Long,Lat
0,Tamale West Hospital,1791548336,hospital,NaN,-0.850863,9.401925
1,Ho Municipal Hospital,2162648419,hospital,NaN,0.468415,6.609965
2,Aburaso Health center,4621957790,hospital,NaN,-1.676991,6.661184
3,Nyaho Medical Centre,5755779938,hospital,NaN,-0.203939,5.551490
4,OPD,6481545351,hospital,NaN,-0.256215,6.097301


### Merge with surrounding facility numbers

In [20]:
def find_grid_index(x,y,longs,lats,x_min=None,x_max=None,y_min=None,y_max=None):
    """
    DESCRIPTION: This function helps to find the index of the longitudes and 
    latitudes in the given longitude/latitude lists that is closest to the 
    given point (x,y) 
    
    INPUT -
    x: latitude of the given point
    y: longtidue of the given point
    longs: reference list of longitudes
    lats: reference list of latitudes
    x_min, x_max: min/max boundaries of longitude
    y_min, y_max: min/max boundaries of latitude
    
    OUTPUT -
    Index of the closest longitude/latitude to point (x,y)
    """
    if x_min is None and x_max is None:
        x_l = longs[0]- (longs[1]-longs[0])
        x_r = longs[-1] + (longs[-1]-longs[-2])
        x_min = min(x_l, x_r)
        x_max = max(x_l, x_r)
    if x < x_min or x > x_max:
        return -1,-1
    
    if y_min is None and y_max is None:
        y_l = lats[0]- (lats[1]-lats[0])
        y_r = lats[-1] + (lats[-1]-lats[-2])
        y_min = min(y_l, y_r)
        y_max = max(y_l, y_r)
    if y < y_min or y > y_max:
        return -1,-1
    
    ncol = np.argmin(abs(longs-x)) 
    nrow = np.argmin(abs(lats-y))
    if nrow<0 or nrow>=(len(lats)-1) or ncol<0 or ncol>=(len(longs)-1):
        return -1, -1
    else:
        return nrow,ncol

In [21]:
long_list = ref_da.coords['x'].values
lat_list = ref_da.coords['y'].values
long_list = np.append(long_list, long_list[-1]+ref_da.rio.resolution()[0])
lat_list = np.append(lat_list, lat_list[-1]+ref_da.rio.resolution()[1])
grid_index = fac_df.apply(lambda row: find_grid_index(row['Long'],row['Lat'],long_list,lat_list), axis=1)
grid_index
fac_df['grid_lat'] = grid_index.apply(lambda x:lat_list[x[0]])
fac_df['grid_long'] = grid_index.apply(lambda x:long_list[x[1]])
fac_df.head(10)

,Facility name,osm_id,Amenity,Ownership,Long,Lat,grid_lat,grid_long
0,Tamale West Hospital,1791548336,hospital,NaN,-0.850863,9.401925,9.403750,-0.847083
1,Ho Municipal Hospital,2162648419,hospital,NaN,0.468415,6.609965,6.612084,0.469583
2,Aburaso Health center,4621957790,hospital,NaN,-1.676991,6.661184,6.662084,-1.680417
3,Nyaho Medical Centre,5755779938,hospital,NaN,-0.203939,5.551490,5.553750,-0.205417
4,OPD,6481545351,hospital,NaN,-0.256215,6.097301,6.095417,-0.255417
5,Emergency Ward,6481545556,hospital,NaN,-0.256452,6.097372,6.095417,-0.255417
6,ICU block,6481545557,hospital,NaN,-0.257740,6.097077,6.095417,-0.255417
7,Main Theatre,6481546822,hospital,NaN,-0.257548,6.096995,6.095417,-0.255417
8,Pharmacy,6481547066,hospital,NaN,-0.256166,6.097560,6.095417,-0.255417
9,Laboratory,6481547290,hospital,NaN,-0.256649,6.097459,6.095417,-0.255417


In [22]:
fac_df['grid_lat_5digits'] = np.round(fac_df['grid_lat'],5)
fac_df['grid_long_5digits'] = np.round(fac_df['grid_long'],5)

In [23]:
# read in facility numbers csv
df1 = pd.read_csv(os.path.join(data_out_path, "fac_2022_0824.csv"), index_col=0)
# df1 = df1[~df1.isnull().any(axis=1)].copy()

df1["Lat_5digits"] = np.round(df1["Lat"], 5)
df1["Long_5digits"] = np.round(df1["Long"], 5)
df1 = df1.drop(["Long", "Lat"], axis=1)

fac_df = fac_df.merge(
    df1,
    how="inner",
    left_on=["grid_lat_5digits", "grid_long_5digits"],
    right_on=["Lat_5digits", "Long_5digits"],
)
fac_df = fac_df.drop(
    [
        "Lat_5digits",
        "Long_5digits",
        "fac_1_flag",
        "fac_3_flag",
        "fac_5_flag",
        "fac_1-3",
        "fac_3-5",
        "fac_5-11",
        "fcr_5"
    ],
    axis=1,
)

fac_df.head()


,Facility name,osm_id,Amenity,Ownership,Long,Lat,grid_lat,grid_long,grid_lat_5digits,grid_long_5digits,fac_1_cnt,fac_3_cnt,fac_5_cnt,fac_11_cnt,fac_11_flag
0,Tamale West Hospital,1791548336,hospital,NaN,-0.850863,9.401925,9.403750,-0.847083,9.40375,-0.84708,1.0,9.0,16.0,41.0,True
1,Ho Municipal Hospital,2162648419,hospital,NaN,0.468415,6.609965,6.612084,0.469583,6.61208,0.46958,1.0,7.0,11.0,13.0,True
2,Aburaso Health center,4621957790,hospital,NaN,-1.676991,6.661184,6.662084,-1.680417,6.66208,-1.68042,0.0,0.0,0.0,11.0,True
3,Nyaho Medical Centre,5755779938,hospital,NaN,-0.203939,5.551490,5.553750,-0.205417,5.55375,-0.20542,4.0,23.0,53.0,156.0,True
4,OPD,6481545351,hospital,NaN,-0.256215,6.097301,6.095417,-0.255417,6.09542,-0.25542,12.0,23.0,28.0,30.0,True


In [24]:
fac_df

,Facility name,osm_id,Amenity,Ownership,Long,Lat,grid_lat,grid_long,grid_lat_5digits,grid_long_5digits,fac_1_cnt,fac_3_cnt,fac_5_cnt,fac_11_cnt,fac_11_flag
0,Tamale West Hospital,1791548336,hospital,NaN,-0.850863,9.401925,9.403750,-0.847083,9.40375,-0.84708,1.0,9.0,16.0,41.0,True
1,Ho Municipal Hospital,2162648419,hospital,NaN,0.468415,6.609965,6.612084,0.469583,6.61208,0.46958,1.0,7.0,11.0,13.0,True
2,Aburaso Health center,4621957790,hospital,NaN,-1.676991,6.661184,6.662084,-1.680417,6.66208,-1.68042,0.0,0.0,0.0,11.0,True
3,Nyaho Medical Centre,5755779938,hospital,NaN,-0.203939,5.551490,5.553750,-0.205417,5.55375,-0.20542,4.0,23.0,53.0,156.0,True
4,OPD,6481545351,hospital,NaN,-0.256215,6.097301,6.095417,-0.255417,6.09542,-0.25542,12.0,23.0,28.0,30.0,True
5,Emergency Ward,6481545556,hospital,NaN,-0.256452,6.097372,6.095417,-0.255417,6.09542,-0.25542,12.0,23.0,28.0,30.0,True
6,ICU block,6481545557,hospital,NaN,-0.257740,6.097077,6.095417,-0.255417,6.09542,-0.25542,12.0,23.0,28.0,30.0,True
7,Main Theatre,6481546822,hospital,NaN,-0.257548,6.096995,6.095417,-0.255417,6.09542,-0.25542,12.0,23.0,28.0,30.0,True
8,Pharmacy,6481547066,hospital,NaN,-0.256166,6.097560,6.095417,-0.255417,6.09542,-0.25542,12.0,23.0,28.0,30.0,True
9,Laboratory,6481547290,hospital,NaN,-0.256649,6.097459,6.095417,-0.255417,6.09542,-0.25542,12.0,23.0,28.0,30.0,True


### Merge with GDP

In [25]:
# read in facility numbers csv
df2 = pd.read_csv(os.path.join(data_out_path, "GDP_2022_0824.csv"), index_col=0)
# df1 = df1[~df1.isnull().any(axis=1)].copy()

df2['Lat_5digits'] = np.round(df2['Lat'],5)
df2['Long_5digits'] = np.round(df2['Long'],5)
df2 = df2.drop(["Long", "Lat"], axis=1)

fac_df = fac_df.merge(df2, how='inner',left_on=['grid_lat_5digits','grid_long_5digits'],right_on=['Lat_5digits','Long_5digits'])
fac_df = fac_df.drop(['Lat_5digits','Long_5digits'],axis=1)
# fac_df = fac_df.rename({'Lat':'grid_lat','Long':'grid_lng'},axis=1)
fac_df.head()

,Facility name,osm_id,Amenity,Ownership,Long,Lat,grid_lat,grid_long,grid_lat_5digits,grid_long_5digits,...,GDP_5,GDP_11,GDP_1_cnt,GDP_3_cnt,GDP_5_cnt,GDP_11_cnt,GDP_1-3,GDP_3-5,GDP_5-11,gdp_rating
0,Tamale West Hospital,1791548336,hospital,NaN,-0.850863,9.401925,9.403750,-0.847083,9.40375,-0.84708,...,7.471393e+08,1.404960e+09,1.0,9.0,25.0,103.0,3.670391e+07,2.567844e+07,8.433603e+06,A
1,Ho Municipal Hospital,2162648419,hospital,NaN,0.468415,6.609965,6.612084,0.469583,6.61208,0.46958,...,5.234166e+08,7.147309e+08,1.0,9.0,24.0,78.0,3.243642e+07,1.409569e+07,3.542858e+06,A
2,Aburaso Health center,4621957790,hospital,NaN,-1.676991,6.661184,6.662084,-1.680417,6.66208,-1.68042,...,4.526427e+08,2.300573e+09,1.0,9.0,25.0,121.0,2.028956e+07,1.750997e+07,1.924927e+07,A
3,Nyaho Medical Centre,5755779938,hospital,NaN,-0.203939,5.551490,5.553750,-0.205417,5.55375,-0.20542,...,1.001801e+09,3.771099e+09,1.0,9.0,22.0,84.0,4.774268e+07,4.396465e+07,4.466610e+07,A
4,OPD,6481545351,hospital,NaN,-0.256215,6.097301,6.095417,-0.255417,6.09542,-0.25542,...,3.101141e+08,7.294132e+08,1.0,9.0,25.0,88.0,1.169458e+07,1.267665e+07,6.655542e+06,A


### Merge with population

In [26]:
# read in population csv
df3 = pd.read_csv(os.path.join(data_out_path, "pop_2022_0824.csv"), index_col=0)

df3['Lat_5digits'] = np.round(df3['Lat'],5)
df3['Long_5digits'] = np.round(df3['Long'],5)
df3 = df3.drop(["Long", "Lat"], axis=1)

fac_df = fac_df.merge(df3, how='inner',left_on=['grid_lat_5digits','grid_long_5digits'],right_on=['Lat_5digits','Long_5digits'])
fac_df = fac_df.drop(['grid_lat', 'grid_long', 'grid_lat_5digits','grid_long_5digits','Lat_5digits','Long_5digits'],axis=1)
# fac_df = fac_df.rename({'Lat':'grid_lat','Long':'grid_lng'},axis=1)
fac_df.head()

,Facility name,osm_id,Amenity,Ownership,Long,Lat,fac_1_cnt,fac_3_cnt,fac_5_cnt,fac_11_cnt,...,pop_3,pop_5,pop_11,pop_1_cnt,pop_3_cnt,pop_5_cnt,pop_11_cnt,pop_1-3,pop_3-5,pop_5-11
0,Tamale West Hospital,1791548336,hospital,NaN,-0.850863,9.401925,1.0,9.0,16.0,41.0,...,27106.7067,67408.99490,1.967177e+05,1.0,9.0,25.0,103.0,24052.9342,40302.28820,129308.69619
1,Ho Municipal Hospital,2162648419,hospital,NaN,0.468415,6.609965,1.0,7.0,11.0,13.0,...,20148.6074,45348.92460,9.372079e+04,1.0,9.0,24.0,78.0,17189.3266,25200.31720,48371.86269
2,Aburaso Health center,4621957790,hospital,NaN,-1.676991,6.661184,0.0,0.0,0.0,11.0,...,27760.3846,78229.85574,6.486553e+05,1.0,9.0,25.0,121.0,23711.9801,50469.47114,570425.42777
3,Nyaho Medical Centre,5755779938,hospital,NaN,-0.203939,5.551490,4.0,23.0,53.0,156.0,...,134564.5640,310169.02300,1.104800e+06,1.0,9.0,22.0,84.0,117512.8140,175604.45900,794630.98320
4,OPD,6481545351,hospital,NaN,-0.256215,6.097301,12.0,23.0,28.0,30.0,...,27026.7504,61723.29253,1.604308e+05,1.0,9.0,25.0,88.0,22787.2860,34696.54213,98707.51078


In [27]:
fac_df.to_csv(os.path.join(data_out_path, "facility_stats.csv"))